<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Kevin", "transactions": [{"transaction-id": 469, "amount": 40}, {"transaction-id": 3196, "amount": 66}, {"transaction-id": 3600, "amount": 73}, {"transaction-id": 4110, "amount": 77}, {"transaction-id": 8152, "amount": 50}, {"transaction-id": 8993, "amount": 68}, {"transaction-id": 9403, "amount": 75}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Kevin", "transactions": [{"transaction-id": 469, "amount": 40}, {"transaction-id": 3196, "amount": 66}, {"transaction-id": 3600, "amount": 73}, {"transaction-id": 4110, "amount": 77}, {"transaction-id": 8152, "amount": 50}, {"transaction-id": 8993, "amount": 68}, {"transaction-id": 9403, "amount": 75}]}\n',
 '{"id": 1, "name": "Victor", "transactions": [{"transaction-id": 3634, "amount": 13}, {"transaction-id": 5858, "amount": -41}, {"transaction-id": 8317, "amount": 97}]}\n',
 '{"id": 2, "name": "Quinn", "transactions": [{"transaction-id": 595, "amount": 788}, {"transaction-id": 1054, "amount": 698}, {"transaction-id": 1723, "amount": 648}, {"transaction-id": 2648, "amount": 724}, {"transaction-id": 2779, "amount": 622}, {"transaction-id": 2832, "amount": 682}, {"transaction-id": 3017, "amount": 677}, {"transaction-id": 3533, "amount": 758}, {"transaction-id": 3554, "amount": 690}, {"transaction-id": 3827, "amount": 693}, {"transaction-id": 3918, "amount": 727}, {

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Kevin',
  'transactions': [{'transaction-id': 469, 'amount': 40},
   {'transaction-id': 3196, 'amount': 66},
   {'transaction-id': 3600, 'amount': 73},
   {'transaction-id': 4110, 'amount': 77},
   {'transaction-id': 8152, 'amount': 50},
   {'transaction-id': 8993, 'amount': 68},
   {'transaction-id': 9403, 'amount': 75}]},
 {'id': 1,
  'name': 'Victor',
  'transactions': [{'transaction-id': 3634, 'amount': 13},
   {'transaction-id': 5858, 'amount': -41},
   {'transaction-id': 8317, 'amount': 97}]},
 {'id': 2,
  'name': 'Quinn',
  'transactions': [{'transaction-id': 595, 'amount': 788},
   {'transaction-id': 1054, 'amount': 698},
   {'transaction-id': 1723, 'amount': 648},
   {'transaction-id': 2648, 'amount': 724},
   {'transaction-id': 2779, 'amount': 622},
   {'transaction-id': 2832, 'amount': 682},
   {'transaction-id': 3017, 'amount': 677},
   {'transaction-id': 3533, 'amount': 758},
   {'transaction-id': 3554, 'amount': 690},
   {'transaction-id': 3827, 'amo

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 92,
  'name': 'Alice',
  'transactions': [{'transaction-id': 127, 'amount': 610},
   {'transaction-id': 373, 'amount': 469},
   {'transaction-id': 406, 'amount': 514},
   {'transaction-id': 415, 'amount': 568},
   {'transaction-id': 556, 'amount': 661},
   {'transaction-id': 594, 'amount': 716},
   {'transaction-id': 661, 'amount': 658},
   {'transaction-id': 930, 'amount': 657},
   {'transaction-id': 964, 'amount': 678},
   {'transaction-id': 993, 'amount': 664},
   {'transaction-id': 1859, 'amount': 545},
   {'transaction-id': 2213, 'amount': 679},
   {'transaction-id': 2636, 'amount': 394},
   {'transaction-id': 2661, 'amount': 763},
   {'transaction-id': 2664, 'amount': 663},
   {'transaction-id': 3136, 'amount': 768},
   {'transaction-id': 3173, 'amount': 506},
   {'transaction-id': 4012, 'amount': 778},
   {'transaction-id': 4092, 'amount': 585},
   {'transaction-id': 4155, 'amount': 700},
   {'transaction-id': 4167, 'amount': 745},
   {'transaction-id': 4380, 'amount': 6

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 42},
 {'name': 'Alice', 'count': 13},
 {'name': 'Alice', 'count': 65},
 {'name': 'Alice', 'count': 1},
 {'name': 'Alice', 'count': 67})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(42, 13, 65, 1, 67)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

42.08433734939759

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 127, 'amount': 610},
  {'transaction-id': 373, 'amount': 469},
  {'transaction-id': 406, 'amount': 514},
  {'transaction-id': 415, 'amount': 568},
  {'transaction-id': 556, 'amount': 661},
  {'transaction-id': 594, 'amount': 716},
  {'transaction-id': 661, 'amount': 658},
  {'transaction-id': 930, 'amount': 657},
  {'transaction-id': 964, 'amount': 678},
  {'transaction-id': 993, 'amount': 664},
  {'transaction-id': 1859, 'amount': 545},
  {'transaction-id': 2213, 'amount': 679},
  {'transaction-id': 2636, 'amount': 394},
  {'transaction-id': 2661, 'amount': 763},
  {'transaction-id': 2664, 'amount': 663},
  {'transaction-id': 3136, 'amount': 768},
  {'transaction-id': 3173, 'amount': 506},
  {'transaction-id': 4012, 'amount': 778},
  {'transaction-id': 4092, 'amount': 585},
  {'transaction-id': 4155, 'amount': 700},
  {'transaction-id': 4167, 'amount': 745},
  {'transaction-id': 4380, 'amount': 692},
  {'transaction-id': 4581, 'amount': 752},
  {'transaction-id': 

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 127, 'amount': 610},
 {'transaction-id': 373, 'amount': 469},
 {'transaction-id': 406, 'amount': 514})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(610, 469, 514)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1090.4842065082546

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 59), ('Alice', 60), ('Alice', 60), ('Alice', 70), ('Bob', 120), ('Bob', 126), ('Bob', 128), ('Bob', 148), ('Charlie', 153), ('Charlie', 179), ('Charlie', 305), ('Dan', 84), ('Dan', 84), ('Dan', 84), ('Dan', 97), ('Edith', 151), ('Edith', 152), ('Edith', 152), ('Edith', 176), ('Frank', 96), ('Frank', 96), ('Frank', 96), ('Frank', 112), ('George', 144), ('George', 144), ('George', 144), ('George', 168), ('Hannah', 120), ('Hannah', 120), ('Hannah', 120), ('Hannah', 140), ('Ingrid', 108), ('Ingrid', 126), ('Ingrid', 216), ('Jerry', 154), ('Jerry', 177), ('Jerry', 305), ('Kevin', 118), ('Kevin', 118), ('Kevin', 119), ('Kevin', 138), ('Laura', 190), ('Laura', 191), ('Laura', 192), ('Laura', 222), ('Michael', 46), ('Michael', 53), ('Michael', 92), ('Norbert', 83), ('Norbert', 83), ('Norbert', 84), ('Norbert', 97), ('Oliver', 120), ('Oliver', 120), ('Oliver', 120), ('Oliver', 140), ('Patricia', 72), ('Patricia', 72), ('Patricia', 72), ('Patricia', 84), ('Quinn', 107), ('Quinn', 108)

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 249), ('Bob', 522), ('Charlie', 637), ('Dan', 349), ('Edith', 631), ('Frank', 400), ('George', 600), ('Hannah', 500), ('Ingrid', 450), ('Jerry', 636), ('Kevin', 493), ('Laura', 795), ('Michael', 191), ('Norbert', 347), ('Oliver', 500), ('Patricia', 300), ('Quinn', 449), ('Ray', 550), ('Sarah', 594), ('Tim', 316), ('Ursula', 442), ('Victor', 367), ('Wendy', 489), ('Xavier', 545), ('Yvonne', 250), ('Zelda', 634)]
CPU times: user 171 ms, sys: 7.69 ms, total: 179 ms
Wall time: 438 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Kevin,"[{'transaction-id': 469, 'amount': 40}, {'tran..."
1,1,Victor,"[{'transaction-id': 3634, 'amount': 13}, {'tra..."
2,2,Quinn,"[{'transaction-id': 595, 'amount': 788}, {'tra..."
3,3,Ray,"[{'transaction-id': 708, 'amount': 489}, {'tra..."
4,4,George,"[{'transaction-id': 101, 'amount': 1079}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 193 ms, sys: 6.21 ms, total: 199 ms
Wall time: 1.23 s


name
Alice      249
Bob        522
Charlie    637
Dan        349
Edith      631
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Kevin', 'amount': 40, 'transaction-id': 469},
 {'id': 0, 'name': 'Kevin', 'amount': 66, 'transaction-id': 3196},
 {'id': 0, 'name': 'Kevin', 'amount': 73, 'transaction-id': 3600})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Kevin,40,469
1,0,Kevin,66,3196
2,0,Kevin,73,3600
3,0,Kevin,77,4110
4,0,Kevin,50,8152


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 138 ms, sys: 15.3 ms, total: 153 ms
Wall time: 883 ms


name
Alice       10479
Bob         12554
Charlie     29966
Dan         11246
Edith       22856
Frank       18579
George      26278
Hannah      23576
Ingrid      21165
Jerry       24622
Kevin       28378
Laura       33015
Michael      7639
Norbert     22426
Oliver      27969
Patricia    11699
Quinn        9908
Ray         20851
Sarah       11397
Tim         10468
Ursula      11975
Victor      12497
Wendy       23389
Xavier      23125
Yvonne      10967
Zelda       32976
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()